<a href="https://colab.research.google.com/github/RummanAli/FYP2022-KnowledgeVision/blob/main/Dermnet_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-fold-Cross-Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
train_folds = []
val_folds = []
for train_index, val_index in skf.split(list_IDs,labels):
  train_folds.append(train_index)
  val_folds.append(val_index)

In [ ]:
folds = []
for train_fold,val_fold in zip(train_folds,val_folds):
  fold = {'train': [],'validation' : []}
  for x in train_fold:
    a = df['patientID'].unique()[x]
    index = df.index[df['patientID']== a].tolist()
    for img in index:
      fold['train'].append(df['imageID'][img])

  for y in val_fold:
    a = df['patientID'].unique()[y]
    index = df.index[df['patientID']== a].tolist()
    for img in index:
      fold['validation'].append(df['imageID'][img])
  folds.append(fold)

# Training

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
import numpy as np
import keras
import tensorflow
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(224,224), n_channels=3,
                 n_classes=23, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img = Image.open(ID)
            img = tf.keras.preprocessing.image.smart_resize(np.array(img),(224,224))
            X[i,] = img

            # Store class
            y[i] = self.labels[ID]
            
        return X, tensorflow.keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
import os
a = os.listdir('drive/MyDrive/archive/train/')
a.sort()
a

['Acne and Rosacea Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Atopic Dermatitis Photos',
 'Bullous Disease Photos',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Eczema Photos',
 'Exanthems and Drug Eruptions',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation',
 'Lupus and other Connective Tissue diseases',
 'Melanoma Skin Cancer Nevi and Moles',
 'Nail Fungus and other Nail Disease',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Seborrheic Keratoses and other Benign Tumors',
 'Systemic Disease',
 'Tinea Ringworm Candidiasis and other Fungal Infections',
 'Urticaria Hives',
 'Vascular Tumors',
 'Vasculitis Photos',
 'Warts Molluscum and other Viral Infections']

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import sklearn
import os
import numpy as np
import numpy as np
image_datasets = {x: datasets.ImageFolder(os.path.join('drive/MyDrive/archive', x)) for x in ['train', 'test']}
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8, shuffle=True, num_workers=4) for x in ['train', 'test']}


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
print(image_datasets['test'].class_to_idx)

{'Acne and Rosacea Photos': 0, 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions': 1, 'Atopic Dermatitis Photos': 2, 'Bullous Disease Photos': 3, 'Cellulitis Impetigo and other Bacterial Infections': 4, 'Eczema Photos': 5, 'Exanthems and Drug Eruptions': 6, 'Hair Loss Photos Alopecia and other Hair Diseases': 7, 'Herpes HPV and other STDs Photos': 8, 'Light Diseases and Disorders of Pigmentation': 9, 'Lupus and other Connective Tissue diseases': 10, 'Melanoma Skin Cancer Nevi and Moles': 11, 'Nail Fungus and other Nail Disease': 12, 'Poison Ivy Photos and other Contact Dermatitis': 13, 'Psoriasis pictures Lichen Planus and related diseases': 14, 'Scabies Lyme Disease and other Infestations and Bites': 15, 'Seborrheic Keratoses and other Benign Tumors': 16, 'Systemic Disease': 17, 'Tinea Ringworm Candidiasis and other Fungal Infections': 18, 'Urticaria Hives': 19, 'Vascular Tumors': 20, 'Vasculitis Photos': 21, 'Warts Molluscum and other Viral Infections': 22}


In [ ]:
img_list = os.listdir('../input/dermnet/train')
img_list.sort()
list_IDs = []
labels = {}
for i,path in enumerate(img_list):
    for img in os.listdir(os.path.join('../input/dermnet/train',path)):
        final = os.path.join('../input/dermnet/train',path,img)
        list_IDs.append(final)
        labels[final] = i

        
img_list = os.listdir('../input/dermnet/test')
img_list.sort()
list_IDs2 = []
labels2 = {}
for i,path in enumerate(img_list):
    for img in os.listdir(os.path.join('../input/dermnet/test',path)):
        final = os.path.join('../input/dermnet/test',path,img)
        list_IDs2.append(final)
        labels2[final] = i


In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal')
])

In [ ]:
class_weight = {0: 840., 1: 1149.,2: 489.,3: 448., 4: 288., 5: 1235., 6: 404.,7: 239.,8: 405.,9: 568.,10: 420.,11: 463.,
                12: 1040., 13: 260., 14: 1405.,15: 431., 16: 1371., 17: 606., 18: 1300.,19: 212.,20: 482.,21: 416.,22: 1086.}

In [ ]:
step = tf.Variable(0, trainable=False)
boundaries = [5, 10,15,20]
values = [0.0001, 0.0001/2, 0.0001/4,0.0001/8,0.0001/16]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)
learning_rate = learning_rate_fn(step)

In [ ]:
import numpy as np
inputs = Input((224,224,3))
x = data_augmentation(inputs)
x = net(x)
x = Flatten()(x)
outputs = Dense(23,activation = 'softmax')(x)
model = Model(inputs, outputs)
model.compile(loss='categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])
model.summary()

In [ ]:
for i,path in enumerate(img_list):
    for img in os.listdir(os.path.join('../input/dermnet/train',path)):
        labels.append(i)
